In [4]:
!pip install parler_tts==0.2.3
!pip install tqdm==4.67.1
!pip install transformers==4.47.0
!pip install soundfile==0.12.1
!pip install torch==2.5.1

### Loading model

In [5]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(device)
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
description_tokenizer = AutoTokenizer.from_pretrained(model.config.text_encoder._name_or_path)

### Defining prompts

In [2]:
prompts = ["कभी-कभी लगता है जैसे ये ज़िंदगी एक पहेली है... हर मोड़ पर एक नया सवाल, एक नया इम्तिहान। हम भागते रहते हैं, कुछ पाने की चाह में, और जो हमारे पास है... उसे भूल जाते हैं। अजीब बात है ना? जितना करीब जाते हैं मंज़िल के, उतनी ही दूर होती दिखती है। कई बार तो ऐसा लगता है कि हम ख़ुद से ही भाग रहे हैं। दूसरों को खुश करने में, अपनी असल खुशी कहाँ छूट गई, पता ही नहीं चला। पर शायद... ज़िंदगी का असली मतलब रास्ते में छुपा है, न कि उस मंज़िल में। बस एक बात समझ ली है — गिरना, संभलना, और फिर से खड़ा होना... यही असली जीत है। डर लगे तो रुक जाना ठीक है, लेकिन हार मान लेना... ये कभी ठीक नहीं। क्यूँकि हर नई शुरुआत, कहीं न कहीं... एक पुराने अंत से ही निकलती है।",
           "अजीब है ना ये ज़िंदगी? हर कोई भाग रहा है... कोई नाम के पीछे, कोई पैसे के पीछे, तो कोई प्यार के पीछे। और हम सोचते हैं कि जो मिल जाएगा, सब ठीक हो जाएगा। लेकिन जो सच में जरूरी है... वो कभी हमारी लिस्ट में होता ही नहीं — सुकून। कभी खुद से सवाल किया है कि आख़िरी बार कब अपने लिए जिया था? बिना किसी वजह के... बिना किसी उम्मीद के? सुबह सूरज की रोशनी को महसूस किया था? बारिश की बूंदों को हथेली पर थामा था? नहीं किया ना? क्योंकि हम हर चीज़ को प्लान करने में लगे हैं। पर ज़िंदगी ना... ये किसी प्लान की मोहताज नहीं। ये वही पल है, जो अभी हमारे सामने है। इसे जी लो... क्योंकि कल का वादा किसी ने नहीं किया। और शायद, यही पल... सबसे कीमती है।",
           "জীবনটা বড় অদ্ভুত, তাই না? প্রতিদিন এক নতুন দৌড়, নতুন প্রতিযোগিতা। আমরা সবাই ছুটছি... সাফল্যের পিছনে, টাকার পিছনে, কিংবা ভালোবাসার পিছনে। কিন্তু এই দৌড়ে কখনো নিজেকে প্রশ্ন করেছ? সত্যি কী চাও? আমরা ভাবি, একটা লক্ষ্য ছুঁলেই সব ঠিক হয়ে যাবে। কিন্তু সেই লক্ষ্যে পৌঁছে কি কখনো শান্তি পেয়েছি? মনে হয় না... কারণ সত্যিকারের আনন্দ লুকিয়ে আছে এই পথ চলার মাঝেই। একটু থেমে দেখো... আকাশটা ঠিক কতটা নীল! একমুঠো বাতাস গালে ছুঁয়ে গেলে কী শান্তি লাগে! আমাদের প্রাপ্তির হিসেব করতে করতে আমরা এই ছোট্ট মুহূর্তগুলো মিস করে ফেলি। অথচ, এই মুহূর্তগুলোই তো আসল জীবন। বেঁচে থাকো, প্রাণ ভরে... কারণ কাল যে কেমন হবে, তা তো কেউ জানে না।",
           "నువ్వు నా జీవితంలోకి వచ్చిన క్షణం నుంచి ప్రతీదీ మారిపోయింది. నా గుండె నువ్వు నవ్విన ప్రతీసారీ కొత్తగా కొట్టుకుంటుంది. నువ్వు చూస్తే చాలు, నా ప్రపంచం నిలిచిపోతుంది. ఏదైనా చెప్పాలనిపించినా మాటలు రావు, కానీ నువ్వు నా ఎదురుగా ఉంటే నా మనసంతా గోసలు చెప్తుంది. నువ్వు రాకపోతే నా రోజు అసంపూర్ణం. నీ మెసేజ్ కోసం చూస్తూ రోజంతా కూర్చుంటా. నీ పేరు వినగానే నా ముఖం నిండా చిరునవ్వులు చిందిస్తాయి. ఏమీ అర్థం కాని చిన్న విషయాలు కూడా నీతో పంచుకుంటే పర్వతాలను పరిగెత్తినంత సంతోషంగా ఉంటుంది. ప్రతీ రాత్రి పడుకోబోయేటప్పుడు ఒకే ఒక్క ఆలోచన — నీతో ఒక రోజు నీ భుజాన తల వాల్చుకుని జీవితాన్ని ఆస్వాదించాలి. నీతో మాట్లాడే ప్రతి నిమిషం ఒక సంపదలా అనిపిస్తుంది. నువ్వు చల్లని గాలివా, నా బతుకును తాకే పసందైన వర్షపు తొలి చినుకులా మారిపోయావు. ఒక సారి అడగు... నేను అంత దూరం నడుస్తా, ఏ గమ్యం లేకున్నా, నిన్ను చేరడానికి. నా జీవితానికి అర్థం నువ్వే, నా ప్రతీ గడిచిన క్షణం నీకోసమే. నీ ప్రేమే నా శ్వాస, నా ప్రాణం. ఎప్పుడు చెప్పగలనో ఆ రోజు కోసం ఆత్రంగా ఎదురు చూస్తున్నా.",
           "ये ज़िंदगी भी अजीब है ना, समझ में ही नहीं आता कि जा कहाँ रही है। कितना कुछ करना चाहता हूँ, और कितना अधूरा रह जाता है। कभी-कभी लगता है कि इंसान मीठा सपना देख रहा है, लेकिन उस सपने को हकीकत में बदलने की हिम्मत नहीं जुटा पाता। ज़िंदगी में जोश है, उम्मीदें हैं... लेकिन कहीं अंदर एक छोटी सी कमजोरी है, जो बार-बार पीछे खींच लेती है। लेकिन अब वक्त आ गया है... खुद के लिए खड़े होने का! कभी सोचा है, दुनिया हमें कैसे याद रखेगी? नहीं, मुझे कोई महान नहीं बनना। बस... एक ऐसा निशान छोड़ना है कि किसी दिन किसी के चेहरे पर मेरे लिए एक मुस्कान आ जाए। मुझे बदलाव करना है। मेरी कमजोरियाँ मेरी ताकत बनेंगी... और जब कोई पीछे हटने को कहेगा, तब मैं मुस्कुराकर कहूँगा, मैं तो आगे बढ़ चुका हूँ। बस... यही है मेरी कहानी। साधारण, लेकिन असरदार।",
           "तिमीलाई पहिलो पटक देखेको दिनदेखि मेरो जीवन बदलियो। तिम्रो मुस्कानले मेरो मुटुको कुनाकुनासम्म छोएको थियो। अचम्म लाग्छ, एक व्यक्तिले कसरी अर्को व्यक्तिको संसारै रंगीन बनाउन सक्छ! तिमी बिना मेरा दिन अधुरा लाग्छन्। तिम्रो आवाज नसुनी निद्रा नै लाग्दैन। कहिले कहिँ आँखा चिम्लेर भविष्यको कल्पना गर्छु—जहाँ म र तिमी एकसाथ छौं। साना-साना कुरामा हाँस्दै, एकअर्काको सपना बाँड्दै... त्यो क्षण कति सुन्दर होला, हैन? तिमी मेरो लागि केवल एक नाम होइनौं, तिमी मेरो संसार हौं। तिम्रो खुशी नै मेरो सबैभन्दा ठूलो चाहना हो। मसँग सायद थोरै शब्दहरू छन् तर धेरै माया छ। तिमीलाई यो कुरा बुझाउन हर प्रयास गर्नेछु। कुनै दिन यो मनको कुरा शब्दमा भन्न सकूँला, तर तबसम्म... तिम्रा आँखा हेरेर नै म आफ्नो भावना देखाउनेछु। किनभने, मेरो हृदयको कथा तिम्रो मुस्कानमा हराउनेछ।",
           "ਜਦੋਂ ਤੈਨੂੰ ਪਹਿਲੀ ਵਾਰ ਵੇਖਿਆ ਸੀ, ਦਿਲ ਜਿਵੇਂ ਇੱਕ ਹੀ ਵਾਰ ਵਿੱਚ ਬੇਕਾਬੂ ਹੋ ਗਿਆ। ਤੇਰੀਆਂ ਅੱਖਾਂ ਵਿੱਚ ਕੋਈ ਜਾਦੂ ਸੀ... ਤੇਰੀ ਮਸਕਾਨ ਦੇ ਅੱਗੇ ਤਾਂ ਸਾਰੀ ਦੁਨੀਆ ਫਿੱਕੀ ਲੱਗਦੀ ਹੈ। ਮੈਨੂੰ ਪਤਾ ਨਹੀਂ ਕਿਉਂ, ਪਰ ਤੈਨੂੰ ਦੇਖ ਕੇ ਹੀ ਲੱਗਦਾ ਹੈ ਕਿ ਮੇਰੀ ਹਰ ਖੁਸ਼ੀ ਤੇਰੇ ਨਾਲ ਹੀ ਜੁੜੀ ਹੋਈ ਹੈ। ਤੇਰੇ ਨਾਲ ਗੱਲਾਂ ਕਰਦਿਆਂ ਸਮਾਂ ਕਿਵੇਂ ਲੰਘ ਜਾਂਦਾ ਹੈ, ਪਤਾ ਹੀ ਨਹੀਂ ਲੱਗਦਾ। ਤੂੰ ਹੱਸੇ ਤਾਂ ਮੇਰਾ ਦਿਨ ਬਣ ਜਾਂਦਾ ਹੈ। ਤੇਰੇ ਕੋਲ ਖੜ੍ਹਾ ਹੋਣ ਦੀ ਚਾਹਤ ਹੀ ਮੈਨੂੰ ਹਮੇਸ਼ਾ ਅੱਗੇ ਵਧਣ ਦਾ ਹੌਸਲਾ ਦਿੰਦੀ ਹੈ। ਪਰ ਇੱਕ ਗੱਲ ਦੱਸਾ? ਕਈ ਵਾਰ ਡਰ ਲੱਗਦਾ ਹੈ, ਜਿਵੇਂ ਇਹ ਸਭ ਸਿਰਫ ਇੱਕ ਖ਼ੁਵਾਬ ਹੋਵੇ। ਪਰ ਫਿਰ ਮੈਂ ਆਪਣੇ ਆਪ ਨੂੰ ਕਹਿੰਦਾ ਹਾਂ — ਜੇ ਇਹ ਖ਼ੁਵਾਬ ਵੀ ਹੈ, ਤਾਂ ਮੈਂ ਇਹ ਖ਼ੁਵਾਬ ਸਦੀਵਾਂ ਦੇਖਣਾ ਚਾਹੁੰਦਾ ਹਾਂ। ਤੂੰ ਮੇਰੀ ਜ਼ਿੰਦਗੀ ਦਾ ਹਿੱਸਾ ਨਹੀਂ, ਤੂੰ ਮੇਰੀ ਜ਼ਿੰਦਗੀ ਹੈ। ਤੂੰ ਹਮੇਸ਼ਾ ਖੁਸ਼ ਰਹੇ, ਇਹੀ ਮੇਰੀ ਸਭ ਤੋਂ ਵੱਡੀ ਦुआ ਹੈ। ਕਦੇ ਮੌਕਾ ਮਿਲਿਆ, ਤਾਂ ਆਪਣੇ ਦਿਲ ਦੀ ਹਰ ਗੱਲ ਤੈਨੂੰ ਕਹਿ ਦੇਵਾਂਗਾ। ਪਰ ਅੱਜ ਲਈ, ਇਹ ਇੱਕ ਮਿੰਟ ਮੇਰੇ ਜਜ਼ਬਾਤਾਂ ਦਾ ਛੋਟਾ ਹਿੱਸਾ ਹੈ।",
           "हर सुबह एक नया मौका लेकर आती है। एक ऐसा मौका, जब आप खुद को साबित कर सकते हैं। मुझे पता है कि जिंदगी आसान नहीं होती। कभी हार, कभी नाकामी, तो कभी उम्मीदों का टूटना। लेकिन क्या हम इसलिए रुक जाएँ? नहीं। सपने देखने की हिम्मत करो, क्योंकि अगर सपना बड़ा होगा तो मेहनत भी बड़ी करनी पड़ेगी। मुझे पता है रास्ता कठिन होगा, लोग हँसेंगे, सवाल उठाएँगे। लेकिन मेरे पास एक जवाब है—मेहनत। जितना नीचे गिराओगे, उतनी ही ताकत से उठूँगा। हर मुश्किल को पार करूँगा, हर दर्द को सहूँगा, क्योंकि मुझे अपने आप पर यकीन है। लोग कहते हैं कि भाग्य तय करता है कि हमें क्या मिलता है। पर मुझे लगता है, मेहनत भाग्य को भी बदल सकती है। आज अगर मैं संघर्ष कर रहा हूँ, तो कल जीत मेरी होगी। वक्त लगेगा, पसीना बहेगा, शायद आँसू भी आएँ। पर जब मंजिल मिलेगी, तो हर मुश्किल का स्वाद मीठा लगेगा। क्योंकि तब मैं ये नहीं कहूँगा कि मैं जीत गया, बल्कि ये कहूँगा कि मैंने खुद को पा लिया। मेरा असली मुकाबला दूसरों से नहीं, खुद से है। और इस बार, मैं खुद को हराकर ही जीतूँगा।",
           "कभी सोचा था कि दोस्ती हमारी सबसे बड़ी ताकत होगी। हर पल, हर खुशी हमने साथ जी थी। स्कूल की वो छुट्टियां, बेमतलब की बातें, घंटों हंसी... सबकुछ याद है। तुम वो दोस्त थे जिसके बिना मैं कुछ भी नहीं था। पर पता नहीं कब चीजें बदल गईं। एक छोटी-सी बात ने इतना बड़ा मोड़ कैसे ले लिया? वो भरोसा जो पत्थर की तरह मजबूत था, अचानक रेत की तरह बिखर गया। मुझे लगा था कि दोस्ती हर जंग जीत सकती है, पर शायद मैं गलत था। मैंने तुम्हें मनाने की कोशिश की, समझाने की भी। पर जवाब में सिर्फ खामोशी मिली। तुम्हारी वो चुप्पी आज भी कानों में गूंजती है। अब सोचता हूँ, क्या हम फिर से वैसे हो सकते हैं जैसे पहले थे? क्या पुरानी बातों को भूलकर फिर से दोस्त बन सकते हैं? या ये दर्द हमेशा रहेगा? शायद वक्त जवाब देगा, या शायद कभी नहीं। लेकिन जो भी हो, मैं दुआ करता हूँ कि तुम खुश रहो... क्योंकि भले ही हमारी दोस्ती खत्म हो गई हो, तुम्हारी यादें हमेशा मेरे दिल में रहेंगी।",
           "तुम्हें देखता हूँ, तो ऐसा लगता है जैसे वक्त ठहर गया है। पहली बार जब तुम्हें देखा था, तब भी दिल ने कुछ महसूस किया था। पर मैंने उसे नज़रअंदाज़ कर दिया। दोस्ती की सीमा को कभी लांघने की हिम्मत नहीं हुई। पर अब दिल को समझाना मुश्किल हो गया है। हर बार जब तुम मुस्कुराती हो, लगता है जैसे मेरी पूरी दुनिया रोशनी से भर गई है। जब तुम उदास होती हो, तो दिल में अजीब-सा खालीपन महसूस होता है। मैं नहीं जानता कि ये कब हुआ, कैसे हुआ... पर हुआ ज़रूर है। तुम्हारे बिना ज़िंदगी अधूरी लगती है। मैं जानता हूँ, ये कह देना आसान नहीं है। डर है कि कहीं तुम्हें खो न दूँ। पर सच तो ये है कि अगर मैंने कहा नहीं, तो शायद मैं खुद को कभी माफ़ नहीं कर पाऊंगा। तो आज हिम्मत करके कहता हूँ—मैं तुमसे प्यार करता हूँ। तुम्हारा जवाब कुछ भी हो, मेरी दोस्ती कभी नहीं बदलेगी। बस इतना चाहता हूँ कि तुम्हें हमेशा खुश देखूँ। क्योंकि भले ही हमारी दोस्ती खत्म हो गई हो, तुम्हारी यादें हमेशा मेरे दिल में रहेंगी।",
           "मुझे गर्व है कि मैं इस देश का हिस्सा हूँ। ये धरती मेरी पहचान है, और इसकी मिट्टी में मेरी आत्मा बसी है। हर सुबह जब तिरंगा लहराता है, तो दिल में एक अजीब-सी ऊर्जा महसूस होती है। हाँ, हम एक परफेक्ट देश नहीं हैं। समस्याएं हैं—भ्रष्टाचार, बेरोजगारी, गरीबी। लेकिन क्या इसका मतलब ये है कि हम हार मान लें? नहीं! हमें अपने हिस्से की जिम्मेदारी उठानी होगी। हर छोटी कोशिश से बदलाव आता है। इतिहास गवाह है कि जब युवा एकजुट होते हैं, तो क्रांतियाँ होती हैं। अगर हम ठान लें कि हमें इस देश को बेहतर बनाना है, तो कोई ताकत हमें रोक नहीं सकती। ये देश सिर्फ जमीन का टुकड़ा नहीं है। ये हमारी संस्कृति, हमारे संस्कार, और हमारे सपनों का प्रतीक है। मुझे यकीन है कि एक दिन ये देश फिर से विश्वगुरु बनेगा। लेकिन उसके लिए हमें आज से काम शुरू करना होगा। छोटी-छोटी चीजों से फर्क पड़ता है। ईमानदारी, मेहनत, और देशभक्ति—यही हमारी असली ताकत है।",
           "यार, ये जिंदगी भी बड़ी मजेदार चीज़ है। सुबह उठते ही ऐसा लगता है जैसे अलार्म हमारे सपनों से बदला लेने आया हो। फिर ऑफिस जाना एक मिशन की तरह लगता है। मेट्रो में ऐसी भीड़ होती है कि लगता है हम कोई जिम का क्राउडेड सेक्शन पार कर रहे हैं। और ऑफिस की मीटिंग्स? बस पूछो मत! हर बार ऐसा लगता है जैसे कोई गुप्त परीक्षा चल रही हो। बॉस के सवाल सुनते ही ऐसा लगता है कि दिमाग ब्लैंक हो गया। और वो वीकेंड का इंतजार? वो तो किसी त्यौहार से कम नहीं लगता। लेकिन जैसे ही रविवार की रात आती है, दिल फिर से रोने लगता है। पर सोचो, अगर ये सब न हो, तो जिंदगी कितनी बोरिंग हो जाएगी। ये नाटक, ये ड्रामा ही तो हमें जिंदा रखता है। कभी-कभी सोचता हूँ, हम सब एक रियलिटी शो में जी रहे हैं, जहां हर दिन नया एपिसोड होता है।",
           "कभी-कभी मैं खुद से सवाल करता हूँ—क्या मैं सही रास्ते पर हूँ? क्या मैं वही इंसान हूँ जो बनने का सपना देखा था? हम सब दौड़ में लगे रहते हैं, मंजिल की तलाश में। पर क्या हमने कभी सोचा है कि वो मंजिल क्या है? बचपन में चीजें कितनी आसान लगती थीं। सपने बड़े थे, डर कम। पर जैसे-जैसे बड़े हुए, जिम्मेदारियाँ बढ़ती गईं। और हम भूल गए कि असली खुशी छोटी-छोटी चीजों में है। एक मुस्कान, एक सच्ची तारीफ, या अपने आप को समय देना। ये सब मायने रखता है। अब मैं कोशिश कर रहा हूँ कि खुद से जुड़ सकूँ। बाहर की दौड़ में जीतने से पहले, अंदर की शांति पाना जरूरी है। क्योंकि असली जीत वही है—जब आप खुद से प्यार करना सीख जाएं।",
           "எப்போ நினைத்தாலும் நம்மைப் பற்றித்தான் நினைத்துப் பார்ப்பேன். நம்முடைய நட்பு எப்படி ஆரம்பித்தது, எவ்வளவு மகிழ்ச்சியாக இருந்தது! பள்ளியில் ஒரே கடைசி பக்கம் வரை நாம்தான் சேர்ந்திருந்தோம். எதையும் பகிர்ந்து கொள்ளும் அளவிற்கு நம்முடைய நட்பு வலுவாக இருந்தது. ஆனா, இப்போ அந்த நட்பு எங்கே இருக்கிறது? எப்போ எதோ ஒரு பிரச்சனையால் பேசாமலேயே வெறுப்பாகி போனோம். நீங்க சொன்னதெல்லாம் என்னை நொறுக்கியது. நீங்க என்னை புரிந்துகொள்வீங்கன்னு நான் நம்பினேன். ஆனா, நம்முடைய இடையில இருந்த உறவு நொடியில் மாறிப்போச்சு. இதை மாற்ற முடியுமா? பழைய மாதிரியான நண்பர்கள் ஆக முடியுமா? அப்படியானால், எதையும் மன்னிக்கத் தயார். நட்பு ஒருபோதும் இழக்கக் கூடாதது. என் மனசு இன்னும் அது தான் நினைக்கிறது, அதில் மட்டும் மாற்றமில்லை.",
           "நான் எப்போதுமே உன்னைக் கண்ணால் பார்த்ததுமே என்னோட இதயம் வேகமாக துடிக்க ஆரம்பிக்கிறது. முதலில் அதனை கற்பனை என்று நினைத்தேன். ஆனா அப்படியில்லை. நீ பேசும்போதெல்லாம் என் மனசு அதற்குள் அடங்கி விடுகிறது. உன் சிரிப்புக்காக என்னைப் பல முறை கலாய்க்கலாம். ஆனா அந்த சிரிப்பு என் வாழ்வின் ஒளியாக இருக்கிறது. என்னுடைய அன்றாட வாழ்க்கையில் நீ இல்லாவிட்டால் அது வெறுமையாக இருக்கும். இதுவரை நம்பிக்கையோடு வெளிப்படுத்த முடியாத உணர்ச்சியை இப்போது சொல்வதற்கு துணிவுடன் இருக்கிறேன். நான் உன்னை சிரமப்படுத்த நினைக்கவில்லை. உன் பதில் என்னவாயினாலும், என் நட்பு ஒருபோதும் மாறாது. ஆனால் உன்னிடம் இதை சொல்லாமலிருந்தால், நான் என்னைக் கெட்டவனாக எண்ணுவேன். நீயும் என்னை நேசித்தால் அந்தக் கனவு நிறைவேறும்.",
           "നമ്മളെക്കുറിച്ചാണ് എപ്പോഴും ഞാൻ ഓർക്കുന്നത്. നമുക്ക് ഉണ്ടായിരുന്ന സൗഹൃദം എത്ര വിശുദ്ധമായിരുന്നത്! ഒരുമിച്ചിരിച്ചു ജീവിതത്തിലെ എല്ലാ കാര്യങ്ങളും പങ്കിട്ട നിമിഷങ്ങൾ എത്ര മനോഹരമായിരുന്നു. പക്ഷേ, ഇന്നത് എല്ലാം മാറി. നമ്മളിൽ തോന്നിയ ചെറിയൊരു ഭേദം, ഓരോ വാക്കും തിരിച്ചറിയാതെ ഇടപെട്ടപ്പോൾ അത് വലിയ അകലം സൃഷ്ടിച്ചു. നിനക്ക് മനസ്സിലായില്ല എനിക്ക് എത്ര വേദന തോന്നി. എത്ര ശ്രമിച്ചിട്ടും മനസ്സിന്റെ ആ പഴയ ബന്ധം പുനർജ്ജീവിപ്പിക്കാൻ കഴിയുന്നില്ല. എങ്കിലും ഞാൻ ഇന്നും പ്രതീക്ഷിക്കുന്നു. പഴയ നിമിഷങ്ങൾക്കായി ഞാൻ ഇനിയും കാത്തിരിക്കും. കാരണം, നല്ല സൗഹൃദം ഒരിക്കലും നഷ്ടമാകരുത്.",
           "എത്രനാളായിരിക്കും, ഈ ഒളിഞ്ഞു കിടന്ന നിമിഷങ്ങൾ പാടെ മറക്കാൻ കഴിയാതെ പോയി. ഞാൻ ആദ്യമായി നിന്നെ കണ്ട നിമിഷം, എന്റെ ഹൃദയം വേഗം കൂടിയപ്പോൾ ഞാൻ അതിന് മഹത്തായൊരു കാരണം തിരയുകയായിരുന്നു. എന്നാൽ ഇപ്പോൾ അറിയാം, അതെ—ഞാൻ നിന്നെ സ്നേഹിക്കുന്നു. നിന്റെ ഓരോ ചിരിയിലും എന്റെ ലോകം പ്രകാശിക്കുന്നു. ഞാൻ നിനക്കിത് പറയുന്നുവെന്ന് ഒന്ന് സങ്കല്പിക്കൂ. ഇതുവരെയും ഞാൻ മൗനം പാലിച്ചു. പക്ഷേ ഇനിയതാകുന്നില്ല. നീ എന്തു പറയുമ്പോഴും, ഈ സ്നേഹത്തിന് മറുപടി പറയാൻ എനിക്ക് ധൈര്യമുണ്ട്. ഞങ്ങൾക്ക് ഒരുമിച്ചൊരു ജീവിതമുണ്ടാകുമോ?",
           "I keep thinking about us—about our friendship that felt so pure, so unbreakable. All those moments we spent together, sharing every little detail of our lives, were beautiful. But now, everything has changed. One small misunderstanding, one unspoken word, and suddenly we drifted apart. You didn’t realize how much it hurt. I tried to hold on, to bring things back to what they were, but it feels like it’s slipping further away. Yet, I still hope. Deep inside, I believe in second chances. Because true friendship shouldn’t be lost like this. I’ll wait—for us.",
           "For how long can I keep this feeling hidden? The first time I saw you, my heart skipped a beat, and since then, I’ve been trying to figure out why. Now I know—it’s love. Every time you smile, my world lights up. I’ve imagined this moment so many times, confessing how I feel. But every time, I stayed silent. Not today. Today, I want you to know how much you mean to me. So, will you give us a chance? Will you take this leap with me? But every time, I stayed silent. Fear held me back. Fear that I’d ruin what we have. Fear that you wouldn’t feel the same way. But not today. Today, I can’t hold it in any longer. You mean more to me than you’ll ever know, and I don’t want to live with the regret of never telling you.",
           "Life keeps teaching me lessons I never asked for, yet here I am—learning. I’ve often wondered what’s right and what’s wrong, and every time the answer seems different. Sometimes, the hardest journey is the one within. You question your choices, your beliefs, and your path. But there’s something magical about those moments of clarity. They remind you that growth is messy, but it’s worth it. The most important thing is to find peace within yourself. Everything else will follow. t’s in those quiet moments when no one’s watching, when you’re left alone with your thoughts, that you truly face yourself. You question your choices, your dreams, your purpose. You ask yourself if you’re on the right path, or if you’ve been wandering aimlessly all along. And sometimes, the answers aren’t easy to find."          
          ]
descriptions = ["Divya's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Rohit's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Arjun's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Prakash's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Rani's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Amrita's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Divjot's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Divya's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Rohit's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Rani's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Rohit's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Divya's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Aman's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Jaya's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Jaya's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Harish's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Anjali's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Thoma's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Mary's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ,"Ravi's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
               ]


### Audio file generation loop

In [ ]:
for index, (prompt, description) in enumerate(tqdm(zip(prompts, descriptions), desc="Processing prompts", total=len(prompts))):
    description_input_ids = description_tokenizer(description, return_tensors="pt").to(device)
    prompt_input_ids = tokenizer(prompt, return_tensors="pt").to(device)
    
    generation = model.generate(
        input_ids=description_input_ids.input_ids,
        attention_mask=description_input_ids.attention_mask,
        prompt_input_ids=prompt_input_ids.input_ids,
        prompt_attention_mask=prompt_input_ids.attention_mask
    )
    
    audio_arr = generation.cpu().numpy().squeeze()
    sf.write(f"{index}_indic.wav", audio_arr, model.config.sampling_rate)